In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl

from timeit import default_timer

Using backend: pytorch


In [11]:
x = torch.FloatTensor([[[1, 1, 1], [2, 2, 2]], [[1, 1, 1], [2, 2, 2]]])

In [52]:
torch.cat(x.chunk(2), dim=-1).squeeze(dim=0).shape


torch.Size([2, 4])

In [28]:
x.sum(dim=0)

tensor([[2., 2., 2.],
        [4., 4., 4.]])

In [3]:
x = [1, 1, 1]
y = [2, 2, 2]

z = zip(x, y)

print(z)

In [6]:
x = {f'{i}': 'value' for i in range(10)}

x

{'0': 'value',
 '1': 'value',
 '2': 'value',
 '3': 'value',
 '4': 'value',
 '5': 'value',
 '6': 'value',
 '7': 'value',
 '8': 'value',
 '9': 'value'}

In [8]:
for i in x.values():
    print(i)

value
value
value
value
value
value
value
value
value
value


In [13]:
indicies = torch.tensor([[0, 1, 1],
                         [2, 0, 2]])
values = torch.tensor([3, 4, 5], dtype=torch.float32)

x = torch.sparse_coo_tensor(indicies, values, [2, 4])

y = torch.stack([x for _ in range(4)])

y

tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3],
                       [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
                       [2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2]]),
       values=tensor([3., 4., 5., 3., 4., 5., 3., 4., 5., 3., 4., 5.]),
       size=(4, 2, 4), nnz=12, layout=torch.sparse_coo)

In [16]:
etypes = torch.tensor([1, 3, 4, 5, 7, 8, 12])
weight = torch.randn(20, 20)

def idx_select(weight, etypes):
    x = weight.index_select(0, etypes)

    return x

@torch.jit.script
def idx_select_script(weight, etypes):
    x = weight.index_select(0, etypes)

    return x

idx_total = 0

for _ in range(1_000_000):
    start = default_timer()

    idx_select(weight, etypes)

    stop = default_timer()

    idx_total += stop - start

idx_script_total = 0

for _ in range(1_000_000):
    start = default_timer()

    idx_select_script(weight, etypes)

    stop = default_timer()

    idx_script_total += stop - start

print(f'Index select: {idx_total}')
print(f'Index select script: {idx_script_total}')




Index select: 4.251108700176701
Index select script: 4.873979106079787


In [14]:
x = torch.randn(7, 9)
y = torch.randn(7, 7)

z = y @ x

F.softmax(z.sum(dim=1), dim=0)



tensor([2.0704e-05, 9.7609e-01, 6.0041e-04, 2.8287e-08, 2.1964e-02, 7.3661e-05,
        1.2557e-03])